In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cd "/content/drive/MyDrive/Colab Notebooks/Recommendation System/Anime Datasets"

/content/drive/MyDrive/Colab Notebooks/Recommendation System/Anime Datasets


In [ ]:
ls

anime.csv  rating.csv


#Collaborative Recommendation System

In [ ]:
anime_df = pd.read_csv("anime.csv")
rating_df = pd.read_csv("rating.csv")

In [ ]:
pd.set_option('display.max_columns', None)
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [ ]:
anime_with_rating_df = rating_df.merge(anime_df, on="anime_id")

In [ ]:
anime_with_rating_df.rename(columns = {"rating_x":"user_rating","rating_y":"avg_ragting"}, inplace=True)

In [ ]:
anime_with_rating_df.head()

,user_id,anime_id,user_rating,name,genre,type,episodes,avg_ragting,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
1,3,20,8,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
2,5,20,6,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
3,6,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297
4,10,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297


In [ ]:
anime_with_rating_df = anime_with_rating_df[["anime_id", "name", "user_id", "user_rating",]]

In [ ]:
anime_with_rating_df.head()

,anime_id,name,user_id,user_rating
0,20,Naruto,1,-1
1,20,Naruto,3,8
2,20,Naruto,5,6
3,20,Naruto,6,-1
4,20,Naruto,10,-1


***Anime more than 250 users***

In [ ]:
x = anime_with_rating_df.groupby("name").count()["user_id"]>250
popular_anime = x[x].index

In [ ]:
final_anime_df = anime_with_rating_df[anime_with_rating_df["name"].isin(popular_anime)]

***Users who have voted more than***

In [ ]:
y = anime_with_rating_df.groupby("user_id").count()["user_rating"]>50
otaku_users = y[y].index

In [ ]:
final_anime_df = final_anime_df[final_anime_df["user_id"].isin(otaku_users)]

In [ ]:
final_anime_df.shape

(6812418, 4)

***Remoing user who have not rated but watched***

In [ ]:
final_anime_df = final_anime_df[final_anime_df["user_rating"]!=-1]

In [ ]:
final_anime_df = final_anime_df.drop_duplicates()

In [ ]:
final_anime_df.head()

,anime_id,name,user_id,user_rating
1,20,Naruto,3,8
2,20,Naruto,5,6
5,20,Naruto,21,8
7,20,Naruto,34,9
8,20,Naruto,38,6


In [ ]:
anime_pt = final_anime_df.pivot_table(index="name", columns="user_id", values="user_rating")

In [ ]:
anime_pt.fillna(0,inplace=True)

In [ ]:
anime_pt.shape

(3318, 37224)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(anime_pt)

In [ ]:
similarity

array([[1.        , 0.67401045, 0.52833456, ..., 0.14704274, 0.14246223,
        0.15329261],
       [0.67401045, 1.        , 0.62270805, ..., 0.13649458, 0.1510324 ,
        0.14906349],
       [0.52833456, 0.62270805, 1.        , ..., 0.13135696, 0.14536649,
        0.14071919],
       ...,
       [0.14704274, 0.13649458, 0.13135696, ..., 1.        , 0.51884966,
        0.62343751],
       [0.14246223, 0.1510324 , 0.14536649, ..., 0.51884966, 1.        ,
        0.77339547],
       [0.15329261, 0.14906349, 0.14071919, ..., 0.62343751, 0.77339547,
        1.        ]])

In [ ]:
similarity.shape

(3318, 3318)

In [ ]:
def recommend(anime_name):

  index = np.where(anime_pt.index==anime_name)[0][0]
  recomendation = sorted(list(enumerate(similarity[index])), key= lambda x:x[1], reverse=True)[1:6]

  for i in recomendation:
    print(anime_pt.index[i[0]])

In [ ]:
recommend("Trigun")

Cowboy Bebop
Trigun: Badlands Rumble
Samurai Champloo
Fullmetal Alchemist
FLCL


In [ ]:
import pickle

In [ ]:
pickle.dump(anime_pt, open("anime_pivot_table.pkl","wb"))
pickle.dump(similarity,open("similarity.pkl","wb"))

In [ ]:
anime_pt.index

Index(['&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi',
       '&quot;Bungaku Shoujo&quot; Memoire',
       '&quot;Bungaku Shoujo&quot; Movie', '.hack//G.U. Returner',
       '.hack//G.U. Trilogy', '.hack//G.U. Trilogy: Parody Mode',
       '.hack//Gift', '.hack//Intermezzo', '.hack//Liminality',
       '.hack//Quantum',
       ...
       'ef: A Tale of Memories.', 'ef: A Tale of Memories. - Prologue',
       'ef: A Tale of Memories. - Recollections', 'iDOLM@STER Xenoglossia',
       's.CRY.ed', 'xxxHOLiC', 'xxxHOLiC Kei',
       'xxxHOLiC Movie: Manatsu no Yoru no Yume', 'xxxHOLiC Rou',
       'xxxHOLiC Shunmuki'],
      dtype='object', name='name', length=3318)